# Multithreading

By default, Julia starts with a single thread. We must tell it explicitly to start multiple threads.

#### Environmental variable

On Linux/MacOS:

```bash
export JULIA_NUM_THREADS=4
```

On Windows:

```bash
set JULIA_NUM_THREADS=4
```

Afterwards start julia.

#### Command line argument

```bash
julia -t 4
```

or

```bash
julia --threads 4
```

#### Jupyter kernel

You can also create a *Jupyter kernel* for multithreaded Julia:

```julia
using IJulia
installkernel("Julia (4 threads)", env=Dict("JULIA_NUM_THREADS"=>"4"))
```

We can readily check how many threads we are running:

In [ ]:
Threads.nthreads()

Note that, in contrast to our distributed computing discussion, we will only consider only a single process:

In [ ]:
import Distributed: nprocs

nprocs()

## `Threads.@spawn`

The `Threads.@spawn` macro is very similar to the `Distributed.@spawn` macro. However, instead of spawning tasks on different worker processes (potentially on different machines) it spawns tasks on different threads. Basically, it creates (and immediately returns) a `Task` and puts it onto a todo-list. The scheduler will then dynamically assign these tasks to threads.

Let's import everything we need from `Base.Threads`.

In [ ]:
import Base.Threads: @spawn, nthreads, threadid

In [ ]:
@spawn println("test")

While `Threads.@spawn` returns the task right away, we might have to wait until the task is done and then fetch our result (again, just as for `Distributed.@spawn`).

In [ ]:
t = @spawn begin sleep(3); return 4 end # returns right away
@time fetch(t) # we need to wait until the task is done

Note that we can use (some of) the control flow tools that we've already seen:

In [ ]:
@sync t = @spawn begin sleep(3); return 4 end # only returns the task once it is done
@time fetch(t) # no need to wait, the task is already done

Of course, we can spawn many tasks in a loop.

In [ ]:
for i in 1:2*nthreads()
    @spawn println("Hi, I'm ", threadid())
end

Note that the tasks are **dynamically scheduled**. Some threads do more work (more values of i) than others!

## `@threads`

Roughly speaking, the macro `Threads.@threads` is for multithreading (threads) what `Distributed.@distributed` was for distributed computing (processes).

In [ ]:
import Base.Threads: @threads

In [ ]:
@threads for i in 1:2*nthreads()
    println("Hi, I'm ", threadid())
end

In contrast to `@spawn` above, `@threads` is **statically scheduled**. The iteration range of the for loop is divided equally between the threads. Every thread handles precisely two iterations!

(In a future version of Julia, there will likely be other scheduling policies, for example a `:dynamic` similar to the behavior of `@spawn`.)

In [ ]:
# @threads :static for i in 1:2*nthreads()
#     println("Hi, I'm ", threadid())
# end

### Where is my `pmap` pendant?

Currently, it doesn't exist. But, conceptually, we can implement it as follows:

In [ ]:
tmap(fn, itr) = map(fetch, map(i -> Threads.@spawn(fn(i)), itr))

In [ ]:
tmap(i -> println(i, " ($(threadid()))"), 1:10);

Note, however, that this implementation creates temporary allocations.

## `@threads` vs `@spawn`

#### `@threads`

* rather lightweight, i.e. small overhead
* **statically scheduled**
* good for uniform workload

#### `@spawn`

* more overhead
* **dynamically scheduled**
* handles non-uniform workloads well
* efficient nesting

# Example: filling an array in parallel

Our goal ist to fill an array in parallel using `@threads` and `@spawn`, respectively.

Note that while we had to use things like `SharedArrays` in the case of distributed computing, threads share the same memory! We also don't have to use constructs like `@everywhere` since everything is running in the same julia process.

In [ ]:
import Base.Threads: @threads, @spawn, nthreads, threadid

In [ ]:
function fill_array_threads(a)
    @threads for i in 1:length(a)
        a[i] = threadid()
    end
    return a
end

In [ ]:
a = zeros(nthreads()*10);

In [ ]:
fill_array_threads(a)

And here is a version using `Threads.@spawn`.

In [ ]:
function fill_array_spawn(a)
    @sync for i in 1:length(a)
        @spawn(a[i] = threadid())
    end
    return a
end

In [ ]:
a = zeros(nthreads()*10);
fill_array_spawn(a)

Let's benchmark both variants!

In [ ]:
using BenchmarkTools
@btime fill_array_threads(x) samples=10 setup=(x = zeros(nthreads()*10));
@btime fill_array_spawn(x) samples=10 setup=(x = zeros(nthreads()*10));

As we can see, the `@threads` based implementation is much faster. However, this shouldn't be surprising, given that our tasks are lightweight and uniform.

## Non-uniform workload

In [ ]:
using LinearAlgebra

In [ ]:
function fill_array_threads_nonuniform(a)
    @threads for i in 1:length(a)
        # calculate the squared magnitude of random vectors growing cubically in size
        a[i] = sum(abs2, rand() for j in 1:(i^3))
    end
    return a
end

In [ ]:
a = zeros(nthreads()*10);

In [ ]:
fill_array_threads_nonuniform(a);

In [ ]:
function fill_array_spawn_nonuniform(a)
    @sync for i in 1:length(a)
        Threads.@spawn(a[i] = sum(abs2, rand() for j in 1:(i^3)))
    end
    return a
end

In [ ]:
fill_array_spawn_nonuniform(a);

In [ ]:
using BenchmarkTools
@btime fill_array_threads_nonuniform(x) samples=10 setup=(x = zeros(nthreads()*10));
@btime fill_array_spawn_nonuniform(x) samples=10 setup=(x = zeros(nthreads()*10));

We see that compared to above, the result has flipped: the `@spawn` implementation, due to it's dynamic scheduling, is more efficient.

# Threads require more care: parallel summation (naive)

In [ ]:
function mysum(xs)
    s = zero(eltype(xs))
    for x in xs
        s += x
    end
    return s
end

In [ ]:
function mysum_threaded_naive(xs)
    s = zero(eltype(xs))
    @threads for x in xs
        s += x
    end
    return s
end

In [ ]:
xs = rand(100_000);

In [ ]:
@show sum(xs);
@show mysum(xs);
@show mysum_threaded_naive(xs);

## Parallel summation (divide the work)

In [ ]:
function mysum_threaded(xs)
    b = ceil(Int, length(xs)/nthreads())
    map(sub_xs -> Threads.@spawn(sum(sub_xs)), Iterators.partition(xs, b)) .|> fetch |> sum
end

In [ ]:
@show sum(xs);
@show mysum(xs);
@show mysum_threaded(xs);

In [ ]:
using BenchmarkTools
@btime mysum($xs);
@btime mysum_threaded($xs);

## Parallel summation (atomics)

In [ ]:
import Base.Threads: Atomic, atomic_add!

function mysum_threaded_atomics(xs)
    s = Atomic{eltype(xs)}(zero(eltype(xs)))
    @threads for x in xs
        atomic_add!(s, x)
    end
    return s[]
end

In [ ]:
@show mysum(xs);
@show mysum_threaded_atomics(xs);

In [ ]:
@btime mysum(xs);
@btime mysum_threaded_atomics(xs);
@btime mysum_threaded(xs);

See [Atomic Operations](https://docs.julialang.org/en/v1/manual/parallel-computing/#Atomic-Operations-1) in the Julia doc for more information.

## Nesting

Spawning of tasks can also be nested.

In [ ]:
function threaded_fun()
    x = threadid()
    @spawn println("job1", " (spawned from $x, processed by $(threadid()))")
    @spawn println("job2", " (spawned from $x, processed by $(threadid()))")
    @spawn println("job3", " (spawned from $x, processed by $(threadid()))")
end

for i in 1:nthreads()
    @spawn threaded_fun()
end

In [ ]:
function threaded_fun2()
    x = threadid()
    @threads for i in 1:3
        println("job$i", " (spawned from $x, processed by $(threadid()))")
    end
end

@threads for i in 1:nthreads()
    threaded_fun2()
end